In [1]:
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.stats.api as sms
import pandas as pd

In [2]:
bcf = pd.read_excel("BCF.xlsx")
ps_bart = pd.read_excel("ps_BART.xlsx")

In [3]:
bcf.columns

Index(['empirical_ATE_RMSE', 'empirical_ATE_MAE', 'empirical_ATE_MAPE',
       'empirical_ATE_len', 'empirical_ATE_cover', 'theoretical_CATE_RMSE',
       'theoretical_CATE_MAE', 'theoretical_CATE_MAPE', 'theoretical_CATE_len',
       'theoretical_CATE_cover'],
      dtype='object')

In [4]:
# Calculate the mean and standard deviation for each column
mean_values = bcf.mean()
std_values = bcf.std()

# Print the results
print("Mean of each column:")
print(mean_values)
print("\nStandard deviation of each column:")
print(std_values)

Mean of each column:
empirical_ATE_RMSE        0.341381
empirical_ATE_MAE         0.223698
empirical_ATE_MAPE        0.129912
empirical_ATE_len         0.775066
empirical_ATE_cover       0.865080
theoretical_CATE_RMSE     1.883474
theoretical_CATE_MAE      1.514132
theoretical_CATE_MAPE     0.661718
theoretical_CATE_len      1.736438
theoretical_CATE_cover    0.336454
dtype: float64

Standard deviation of each column:
empirical_ATE_RMSE        0.079447
empirical_ATE_MAE         0.061098
empirical_ATE_MAPE        0.033769
empirical_ATE_len         0.077845
empirical_ATE_cover       0.144273
theoretical_CATE_RMSE     0.870690
theoretical_CATE_MAE      0.719695
theoretical_CATE_MAPE     0.121351
theoretical_CATE_len      0.563191
theoretical_CATE_cover    0.080788
dtype: float64


In [5]:
bcf.head()

,empirical_ATE_RMSE,empirical_ATE_MAE,empirical_ATE_MAPE,empirical_ATE_len,empirical_ATE_cover,theoretical_CATE_RMSE,theoretical_CATE_MAE,theoretical_CATE_MAPE,theoretical_CATE_len,theoretical_CATE_cover
0,0.362399,0.231037,0.126752,0.759170,0.820,1.665163,1.322171,0.701802,1.419376,0.320728
1,0.305657,0.198953,0.114469,0.721541,0.944,1.750306,1.399270,0.599120,1.820283,0.334560
2,0.305295,0.233982,0.150902,0.743243,0.796,1.654948,1.304854,0.553845,1.779995,0.362704
3,0.329944,0.259139,0.145770,0.750385,0.648,1.165092,0.916506,0.447306,1.614522,0.505392
4,0.308609,0.213964,0.119625,0.788909,0.936,1.757952,1.406497,0.642386,1.839654,0.351688


In [6]:
# Calculate RSE and AE for 'empirical_ATE_cover' and 'theoretical_CATE_cover'
bcf_SE_empirical_ATE_cover = (((0.95 - bcf['empirical_ATE_cover'])**2))
bcf_AE_empirical_ATE_cover = (np.abs(0.95 - bcf['empirical_ATE_cover']))

bcf_SE_theoretical_CATE_cover = (((0.95 - bcf['theoretical_CATE_cover'])**2))
bcf_AE_theoretical_CATE_cover = (np.abs(0.95 - bcf['theoretical_CATE_cover']))

In [7]:
ps_bart.columns

Index(['empirical_ATE_RMSE', 'empirical_ATE_MAE', 'empirical_ATE_MAPE',
       'empirical_ATE_len', 'empirical_ATE_cover', 'theoretical_CATE_RMSE',
       'theoretical_CATE_MAE', 'theoretical_CATE_MAPE', 'theoretical_CATE_len',
       'theoretical_CATE_cover'],
      dtype='object')

In [8]:
# Calculate the mean and standard deviation for each column
mean_values = ps_bart.mean()
std_values = ps_bart.std()

# Print the results
print("Mean of each column:")
print(mean_values)
print("\nStandard deviation of each column:")
print(std_values)

Mean of each column:
empirical_ATE_RMSE        0.233472
empirical_ATE_MAE         0.198496
empirical_ATE_MAPE        0.123854
empirical_ATE_len         1.217691
empirical_ATE_cover       0.991840
theoretical_CATE_RMSE     0.700338
theoretical_CATE_MAE      0.604559
theoretical_CATE_MAPE     0.319102
theoretical_CATE_len      2.292547
theoretical_CATE_cover    0.892188
dtype: float64

Standard deviation of each column:
empirical_ATE_RMSE        0.093395
empirical_ATE_MAE         0.095611
empirical_ATE_MAPE        0.059684
empirical_ATE_len         0.099615
empirical_ATE_cover       0.035934
theoretical_CATE_RMSE     0.336516
theoretical_CATE_MAE      0.285505
theoretical_CATE_MAPE     0.070197
theoretical_CATE_len      0.334254
theoretical_CATE_cover    0.110149
dtype: float64


In [9]:
# Calculate RSE and AE for 'empirical_ATE_cover' and 'theoretical_CATE_cover'
ps_bart_SE_empirical_ATE_cover = (((0.95 - ps_bart['empirical_ATE_cover'])**2))
ps_bart_AE_empirical_ATE_cover = (np.abs(0.95 - ps_bart['empirical_ATE_cover']))

ps_bart_SE_theoretical_CATE_cover = (((0.95 - ps_bart['theoretical_CATE_cover'])**2))
ps_bart_AE_theoretical_CATE_cover = (np.abs(0.95 - ps_bart['theoretical_CATE_cover']))

In [10]:
# Function to check normality using Shapiro-Wilk test
def check_normality(data):
    stat, p = stats.shapiro(data)
    return p > 0.05  # If p > 0.05, data is normally distributed

# Function to check homoscedasticity using Levene's test
def check_homoscedasticity(data1, data2):
    stat, p = stats.levene(data1, data2)
    return p > 0.05  # If p > 0.05, variances are equal



# ATE RMSE

In [11]:
i=0
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For empirical_ATE_RMSE
Mann-Whitney U Test:
Statistic: 1833.0, p-value: 1.0179733311632538e-14
Wilcoxon Signed-Rank Test:
Statistic: 219.0, p-value: 2.2131438733610918e-15
Kruskal-Wallis H Test:
Statistic: 59.879934328358104, p-value: 1.0082415016307366e-14
Levene's Test:
Statistic: 3.2926126173855224, p-value: 0.0711056459185347
Brown-Forsythe Test:
Statistic: 3.2926126173855224, p-value: 0.0711056459185347


# ATE MAE

In [12]:
i=1
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For empirical_ATE_MAE
Mann-Whitney U Test:
Statistic: 3674.0, p-value: 0.001200707622920069
Wilcoxon Signed-Rank Test:
Statistic: 1486.0, p-value: 0.0003536982091961691
Kruskal-Wallis H Test:
Statistic: 10.497170149253748, p-value: 0.0011955751009868755
Levene's Test:
Statistic: 13.078969973640229, p-value: 0.00037892403444892987
Brown-Forsythe Test:
Statistic: 13.078969973640229, p-value: 0.00037892403444892987


# ATE MAPE

In [13]:
i=2
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For empirical_ATE_MAPE
Mann-Whitney U Test:
Statistic: 4110.0, p-value: 0.02975064481931883
Wilcoxon Signed-Rank Test:
Statistic: 1999.0, p-value: 0.07051956353395923
Kruskal-Wallis H Test:
Statistic: 4.728955223880575, p-value: 0.02965889610251309
Levene's Test:
Statistic: 21.19330973458123, p-value: 7.4126592437814775e-06
Brown-Forsythe Test:
Statistic: 21.19330973458123, p-value: 7.4126592437814775e-06


# CATE RMSE

In [14]:
i=5
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For theoretical_CATE_RMSE
Mann-Whitney U Test:
Statistic: 288.0, p-value: 1.1474946021218599e-30
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 132.55488955223882, p-value: 1.1313498302525806e-30
Levene's Test:
Statistic: 17.243074116022033, p-value: 4.886149696501008e-05
Brown-Forsythe Test:
Statistic: 17.243074116022033, p-value: 4.886149696501008e-05


# CATE MAE

In [15]:
i=6
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For theoretical_CATE_MAE
Mann-Whitney U Test:
Statistic: 371.0, p-value: 1.1812766826774982e-29
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 127.92621492537319, p-value: 1.1649430828381949e-29
Levene's Test:
Statistic: 15.43149655111718, p-value: 0.00011812169665426297
Brown-Forsythe Test:
Statistic: 15.43149655111718, p-value: 0.00011812169665426297


# CATE MAPE

In [16]:
i=7
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For theoretical_CATE_MAPE
Mann-Whitney U Test:
Statistic: 51.0, p-value: 1.1768599965488024e-33
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 146.22448358208942, p-value: 1.159487138267953e-33
Levene's Test:
Statistic: 13.028111501874019, p-value: 0.00038869622137016065
Brown-Forsythe Test:
Statistic: 13.028111501874019, p-value: 0.00038869622137016065


# ATE len

In [17]:
i=3
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For empirical_ATE_len
Mann-Whitney U Test:
Statistic: 10000.0, p-value: 2.562143669163401e-34
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 149.25373134328356, p-value: 2.523939423990366e-34
Levene's Test:
Statistic: 4.833512302354932, p-value: 0.029070419483477123
Brown-Forsythe Test:
Statistic: 4.833512302354932, p-value: 0.029070419483477123


# CATE len

In [18]:
i=8
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For theoretical_CATE_len
Mann-Whitney U Test:
Statistic: 8671.0, p-value: 3.006918798236869e-19
Wilcoxon Signed-Rank Test:
Statistic: 117.0, p-value: 1.2370784421819017e-16
Kruskal-Wallis H Test:
Statistic: 80.45517014925372, p-value: 2.9737548635326016e-19
Levene's Test:
Statistic: 7.306672904437419, p-value: 0.007467022112151649
Brown-Forsythe Test:
Statistic: 7.306672904437419, p-value: 0.007467022112151649


# ATE Cover SE

In [19]:
group1 = np.array(ps_bart_SE_empirical_ATE_cover)
group2 = np.array(bcf_SE_empirical_ATE_cover)
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For ATE Cover SE")
    print(f"ATE Cover RMSE for ps-BART: {np.sqrt(np.mean(group1))} and BCF: {np.sqrt(np.mean(group2))}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For ATE Cover SE")
    print(f"ATE Cover RMSE for ps-BART: {np.sqrt(np.mean(group1))} and BCF: {np.sqrt(np.mean(group2))}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For ATE Cover SE
ATE Cover RMSE for ps-BART: 0.05503598822588728 and BCF: 0.16678681003005003
Mann-Whitney U Test:
Statistic: 5303.5, p-value: 0.44461379871232865
Wilcoxon Signed-Rank Test:
Statistic: 1557.0, p-value: 0.0008729723464081116
Kruskal-Wallis H Test:
Statistic: 0.5862762628055728, p-value: 0.4438626881172226
Levene's Test:
Statistic: 28.00518375411646, p-value: 3.1947275249990716e-07
Brown-Forsythe Test:
Statistic: 28.00518375411646, p-value: 3.1947275249990716e-07


# ATE Cover AE

In [20]:
group1 = np.array(ps_bart_AE_empirical_ATE_cover)
group2 = np.array(bcf_AE_empirical_ATE_cover)
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For ATE Cover AE")
    print(f"ATE Cover MAE for ps-BART: {(np.mean(group1))} and BCF: {(np.mean(group2))}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For ATE Cover AE")
    print(f"ATE Cover MAE for ps-BART: {(np.mean(group1))} and BCF: {(np.mean(group2))}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For ATE Cover AE
ATE Cover MAE for ps-BART: 0.05140000000000004 and BCF: 0.11415999999999997
Mann-Whitney U Test:
Statistic: 5303.5, p-value: 0.44461379871232865
Wilcoxon Signed-Rank Test:
Statistic: 1645.0, p-value: 0.0024781157785772334
Kruskal-Wallis H Test:
Statistic: 0.5862762628055728, p-value: 0.4438626881172226
Levene's Test:
Statistic: 52.48838280650056, p-value: 9.428949517284728e-12
Brown-Forsythe Test:
Statistic: 52.48838280650056, p-value: 9.428949517284728e-12


# CATE Cover SE

In [21]:
group1 = np.array(ps_bart_SE_theoretical_CATE_cover)
group2 = np.array(bcf_SE_theoretical_CATE_cover)
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For CATE Cover SE")
    print(f"CATE Cover RMSE for ps-BART: {np.sqrt(np.mean(group1))} and BCF: {np.sqrt(np.mean(group2))}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For CATE Cover SE")
    print(f"CATE Cover RMSE for ps-BART: {np.sqrt(np.mean(group1))} and BCF: {np.sqrt(np.mean(group2))}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For CATE Cover SE
CATE Cover RMSE for ps-BART: 0.12391044241257473 and BCF: 0.6187893589818105
Mann-Whitney U Test:
Statistic: 14.0, p-value: 3.899465799359803e-34
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 148.41919191419183, p-value: 3.841480136532977e-34
Levene's Test:
Statistic: 65.41042387115088, p-value: 5.9758477674515e-14
Brown-Forsythe Test:
Statistic: 65.41042387115088, p-value: 5.9758477674515e-14


# CATE Cover AE

In [22]:
group1 = np.array(ps_bart_AE_theoretical_CATE_cover)
group2 = np.array(bcf_AE_theoretical_CATE_cover)
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For CATE Cover AE")
    print(f"CATE Cover MAE for ps-BART: {(np.mean(group1))} and BCF: {(np.mean(group2))}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For CATE Cover AE")
    print(f"CATE Cover MAE for ps-BART: {(np.mean(group1))} and BCF: {(np.mean(group2))}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For CATE Cover AE
CATE Cover MAE for ps-BART: 0.07985335999999998 and BCF: 0.61354608
Mann-Whitney U Test:
Statistic: 14.0, p-value: 3.899465799359803e-34
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 148.41919191419183, p-value: 3.841480136532977e-34
Levene's Test:
Statistic: 0.008210919933091474, p-value: 0.9278908080498265
Brown-Forsythe Test:
Statistic: 0.008210919933091474, p-value: 0.9278908080498265
